In [2]:
import os
import json
import requests
import pandas as pd

In [6]:
projects = requests.get("https://dcc.icgc.org/api/v1/projects?size=1000").json()["hits"]
print "Found {} total ICGC projects".format(len(projects))

projects_with_ssm = [p for p in projects 
                     if "availableDataTypes" in p 
                     and "ssm" in p["availableDataTypes"]
                     and p["ssmTestedDonorCount"] < 100]
print "Found {} with ssm".format(len(projects_with_ssm))

data_types = ["donor", "simple_somatic_mutation.open"]
files = [{"name": "{1}.{0}.tsv.gz".format(p["id"], d),
          "url": "https://dcc.icgc.org/api/v1/download?fn=/release_22/Projects/{0}/{1}.{0}.tsv.gz".format(p["id"], d)}
         for p in projects_with_ssm for d in data_types]
files_to_download = [f for f in files if not os.path.isfile("/data/icgc/{}".format(f["name"]))]
print "Files to download", len(files_to_download)

Found 70 total ICGC projects
Found 20 with ssm
Files to download 0


In [7]:
for project in projects_with_ssm:
    directory = "/data/icgc_extracted/{}".format(project["primaryCountries"][0])
    print "Extracting project {} into {}".format(project["id"], directory)
    if not os.path.exists(directory):
        os.makedirs(directory)

    donors = pd.read_table("/data/icgc/donor.{}.tsv.gz".format(project["id"]))
    variants = pd.read_table("/data/icgc/simple_somatic_mutation.open.{}.tsv.gz".format(project["id"]))
    submissions = variants.groupby(["icgc_donor_id", "icgc_sample_id"])
    
    print "Creating {} submissions".format(len(submissions))

    for donor_sample, variants in submissions:
        with open("{}/{}_{}.json".format(directory, donor_sample[0], donor_sample[1]), "w+") as fields:
            fields.write(json.dumps(donors[donors["icgc_donor_id"] == donor_sample[0]].to_dict(orient='records')[0]))
        variants.to_csv("{}/{}_{}.tsv".format(directory, donor_sample[0], donor_sample[1]))

Extracting project OV-US into /data/icgc_extracted/United States
Creating 88 submissions
Extracting project ALL-US into /data/icgc_extracted/United States
Creating 34 submissions
Extracting project READ-US into /data/icgc_extracted/United States


/opt/conda/envs/python2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Creating 80 submissions
Extracting project RECA-EU into /data/icgc_extracted/European Union
Creating 95 submissions
Extracting project BRCA-FR into /data/icgc_extracted/France
Creating 64 submissions
Extracting project BOCA-FR into /data/icgc_extracted/France
Creating 98 submissions
Extracting project OV-AU into /data/icgc_extracted/Australia


/opt/conda/envs/python2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Creating 115 submissions
Extracting project SKCA-BR into /data/icgc_extracted/Brazil
Creating 70 submissions
Extracting project BOCA-UK into /data/icgc_extracted/United Kingdom
Creating 66 submissions
Extracting project LUSC-KR into /data/icgc_extracted/South Korea


CParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.